In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from neuralhydrology.nh_run import start_run, eval_run, continue_run
from pathlib import Path
import torch
from neuralhydrology.utils.file_system_operation import get_latest_touched_directory
from neuralhydrology.training.train import start_training
from neuralhydrology.utils.config import Config

def get_predictor_list(s):
    perdictor_dict = {
    'p' : ["precipitation_AWAP"],
    'e' : ["et_morton_point_SILO"],
    'pe': ["precipitation_AWAP", "et_morton_point_SILO"]
    }
    return perdictor_dict[s]

In [5]:
from pathlib import Path
import torch

def train_and_evaluate(fold, predictors, config_file):
    # Set up configuration
    cfg = Config(config_file)
    cfg.update_config({
        'dynamic_inputs': get_predictor_list(predictors),
        'train_basin_file': f'basins/twofoldsplit/fold_{fold}_train.txt',
        'test_basin_file': f'basins/twofoldsplit/fold_{fold}_test.txt',
        'validation_basin_file': f'basins/twofoldsplit/fold_{fold}_test.txt',
        'experiment_name': f'spatial_twofold_{fold}_{predictors}',
    })

    # Set device
    cfg.device = f"cuda:0" if torch.cuda.is_available() else "cpu"

    # Start training
    start_training(cfg)

    # Get the latest run directory
    run_dir = get_latest_touched_directory('runs')

    # Evaluate on test set
    ote = eval_run(run_dir=run_dir, period='test')

    # Evaluate on validation set
    ova = eval_run(run_dir=run_dir, period='validation')

    # Evaluate on train set
    otr = eval_run(run_dir=run_dir, period='train')

    return ote, ova, otr


config_file = Path('configs/config.yml')


for predictors in ['p', 'pe', 'e']:
    for fold in [0, 1]:
        _ = train_and_evaluate(fold, predictors, config_file)


2024-01-25 10:34:08,250: Logging to /Users/sho108/projects/neuralhydrology/workflows/camelaus_lstm/runs/spatial_twofold_0_p_2501_103408/output.log initialized.
2024-01-25 10:34:08,250: ### Folder structure created at /Users/sho108/projects/neuralhydrology/workflows/camelaus_lstm/runs/spatial_twofold_0_p_2501_103408
2024-01-25 10:34:08,251: ### Run configurations for spatial_twofold_0_p
2024-01-25 10:34:08,251: experiment_name: spatial_twofold_0_p
2024-01-25 10:34:08,252: train_basin_file: basins/twofoldsplit/fold_0_train.txt
2024-01-25 10:34:08,252: validation_basin_file: basins/twofoldsplit/fold_0_test.txt
2024-01-25 10:34:08,252: test_basin_file: basins/twofoldsplit/fold_0_test.txt
2024-01-25 10:34:08,253: train_start_date: 1981-01-01 00:00:00
2024-01-25 10:34:08,253: train_end_date: 2018-12-31 00:00:00
2024-01-25 10:34:08,254: validation_start_date: 1981-01-01 00:00:00
2024-01-25 10:34:08,254: validation_end_date: 2018-12-31 00:00:00
2024-01-25 10:34:08,254: test_start_date: 1981-01

KeyboardInterrupt: 